# Segmentation of IMU Data

In [ ]:
from scipy.signal import butter, filtfilt
#The Butterworth filter function
def butter_lowpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def apply_lowpass_filter(data, cutoff_freq, fs=1, order=5):
    b, a = butter_lowpass(cutoff_freq, fs, order=order)
    smoothed_data = filtfilt(b, a, data)
    return smoothed_data


In [ ]:
#rename columns

import pandas as pd
import os
import time
def imu_rawprocess(folder_path):
    #Getting all files and folders in a specified directory
    contents = os.listdir(folder_path)
    for item in contents[:]:
        item_path = os.path.join(folder_path, item)   
        
        if os.path.isdir(item_path):
            print('folder：' + item_path)

        item_contents = os.listdir(item_path)
        
        #Traverse the subfolders 'time01' and 'time02' under the 'sample' folder
        for t in item_contents[:]:
            item_path_time = os.path.join(item_path, t) 
            print('folder：'+item_path_time)
                  
            file_contents = os.listdir(item_path_time)   
            
            #if item=='sample01' or item=='sample02':
            for file in file_contents[:4]:
                file_path = os.path.join(item_path_time, file)
                print('file：'+file_path)
                #Import files
                df1 = pd.read_csv(file_path,index_col=False)
                new_columns=['Timestamp','device_name','ACC_X','ACC_Y','ACC_Z','AV_X','AV_Y','AV_Z','Angle_X','Angle_Y','Angle_Z','Mag_X','Mag_Y','Mag_Z','Temperature']
                df1.columns=new_columns
                #re_columns=['Mag_X','Mag_Y','Mag_Z']
                #df1[re_columns] = df1[re_columns]/100
                df1.to_csv("\\Data_new\\Raw\\IMU\\"+item+"\\"+t+"\\"+file[:-4]+'.csv',index=False)
            '''
            else:
                for file in file_contents[:4]:
                    file_path = os.path.join(item_path_time, file)
                    print('file：'+file_path)
                    #Import files
                    df2 = pd.read_csv(file_path,index_col=False)
                    df2 = df2.drop(columns=['片上时间()','四元数0()','四元数1()','四元数2()','四元数3()'])
                    new_columns=['Timestamp','device_name','ACC_X','ACC_Y','ACC_Z','AV_X','AV_Y','AV_Z','Angle_X','Angle_Y','Angle_Z','Mag_X','Mag_Y','Mag_Z','Temperature']
                    df2.columns=new_columns
                    df2.to_csv("\\Data_new\\Raw\\IMU\\"+item+"\\"+t+"\\"+file[:-4]+'.csv',index=False)
            '''
imu_rawprocess("\\Data_new\\Raw\\IMU")             
                

In [ ]:

import pandas as pd
import os
import time

def imu_preprocess(folder_path):
    #Getting all files and folders in a specified directory
    contents = os.listdir(folder_path)
    timestamp_path='\\Data_new\\Raw\\Timestamp'  #The location of the timestamp
    cutoff_frequency = 7.5 
    sampling_frequency = 100  
    filter_order = 4  
    #Traverse the folder 'sample' under the 'IMU' folder and the 'Timestamp' folder
    for item in contents[:]:
        item_path = os.path.join(folder_path, item)   
        #Browse the timestamp folder for the corresponding item
        item_timestamp= os.path.join(timestamp_path,item)
        print('folder：' + item_timestamp)
        
        if os.path.isdir(item_path):
            print('folder：' + item_path)

        item_contents = os.listdir(item_path)
        
        #Traverse the subfolders 'time01' and 'time02' under the 'sample' folder
        for t in item_contents[:]:
            item_path_time = os.path.join(item_path, t) 
            item_timestamp_time= os.path.join(item_timestamp, t) 
            print('folder：'+item_path_time)
            print('folder：'+item_timestamp_time)
            
            
            #Browse the 'timeID' file under the 'sample' folder
            timestamp=pd.read_csv(item_timestamp_time+'.csv')
                  
            file_contents = os.listdir(item_path_time)   
            
            #The format of the first two sample files recorded by the IMU differs slightly from the format of the subsequent samples, so they need to be processed separately.          
            
      
            #Traverse the 'task' files
            for file in file_contents[:4]:
                if file[:-4] == 'task_normal':
                    row = 0
                if file[:-4] == 'task_cup':
                    row = 1
                if file[:-4] == 'task_text':
                    row = 2
                if file[:-4] == 'task_web':
                    row = 3
                
                file_path = os.path.join(item_path_time, file)
                print('file：'+file_path)
                #Import files
                df_1= pd.read_csv(file_path,index_col=False)
                
                #Convert timestamps to numerical format
                for j in range(6):
                    time_array_imu= time.strptime(timestamp.iloc[row,j+1][:-7], "%Y-%m-%d-%H:%M:%S")
                    timestamp_imu = time.mktime(time_array_imu)
                    timestamp_imu= timestamp_imu+float(timestamp.iloc[row,j+1][-7:-3])
                    timestamp.iloc[row,j+1]=timestamp_imu
                
                #Segment different nodes of timestamp records
                start=timestamp.iloc[row,1]
                start_kinect=timestamp.iloc[row,2]
                start_turn=timestamp.iloc[row,3]
                end_turn=timestamp.iloc[row,4]
                end_kinect=timestamp.iloc[row,5]
                end=timestamp.iloc[row,6]
                #Remove the 2m data at the beginning and end
                imu_new = df_1.copy()
                for i in range(0,len(imu_new)):
                    timestamp2 = imu_new.iloc[i,0]
                    if timestamp2 >= start:
                        imu_new.drop(imu_new.head(i).index,inplace=True)
                        break
                imu_new.reset_index(drop=True,inplace=True)

                for i in range(0,len(imu_new)):
                    timestamp2 = imu_new.iloc[i,0]
                    if timestamp2 > end :
                        imu_new.drop(imu_new.tail(len(imu_new)-i).index,inplace=True)
                        break
                #Filtering        
                imu_new.iloc[:,2:11].apply(lambda x: apply_lowpass_filter(x, cutoff_frequency, fs=sampling_frequency, order=filter_order))
                
                new_order = ['Timestamp','device_name','ACC_Z','ACC_X','ACC_Y','AV_Z','AV_X','AV_Y','Angle_Z','Angle_X','Angle_Y','Mag_Z','Mag_X','Mag_Y','Temperature']
                
                imu_new = imu_new[new_order]

                new_columns=['Timestamp','device_name','ACC_X','ACC_Y','ACC_Z','AV_X','AV_Y','AV_Z','Angle_X','Angle_Y','Angle_Z','Mag_X','Mag_Y','Mag_Z','Temperature']
                imu_new.columns = new_columns
                
                #Segmente IMU data into straight and turning phases: go/go_kinect/back/back_kinect/turn
                #go: start ~ start_turn
                
                imu_go = imu_new.copy()
                for i in range(0,len(imu_go)):
                    timestamp2 = imu_go.iloc[i,0]
                    if timestamp2 >= start:
                        imu_go.drop(imu_go.head(i).index,inplace=True)
                        break
                imu_go.reset_index(drop=True,inplace=True)

                for i in range(0,len(imu_go)):
                    timestamp2 = imu_go.iloc[i,0]
                    if timestamp2 > start_turn :
                        imu_go.drop(imu_go.tail(len(imu_go)-i).index,inplace=True)
                        break
                imu_go['Timestamp'] = imu_go['Timestamp'].astype(str)
                
                if len(imu_go)<2:
                    print("错误")     
                #Angle correction
                first_row_value_x = imu_go.loc[0,'Angle_X']
                first_row_value_y = imu_go.loc[0,'Angle_Y']
                first_row_value_z = imu_go.loc[0,'Angle_Z']

                imu_go.loc[:,'Angle_X'] = imu_go['Angle_X'] - first_row_value_x
                imu_go.loc[:,'Angle_Y'] = imu_go['Angle_Y'] - first_row_value_y
                imu_go.loc[:,'Angle_Z'] = imu_go['Angle_Z'] - first_row_value_z

                imu_go.to_csv("\\Data_new\\Processed\\IMU_5m\\"+t+'\\go\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)
                
                #go_kinect: start_kinect ~ start_turn
                imu_go_kinect = imu_new.copy()
                for i in range(0,len(imu_go_kinect)):
                    timestamp2 = imu_go_kinect.iloc[i,0]
                    if timestamp2 >= start_kinect:
                        imu_go_kinect.drop(imu_go_kinect.head(i).index,inplace=True)
                        break
                imu_go_kinect.reset_index(drop=True,inplace=True)
                for i in range(0,len(imu_go_kinect)):
                    timestamp2 = imu_go_kinect.iloc[i,0]
                    if timestamp2 > start_turn :
                        imu_go_kinect.drop(imu_go_kinect.tail(len(imu_go_kinect)-i).index,inplace=True)
                        break
                imu_go_kinect['Timestamp'] = imu_go_kinect['Timestamp'].astype(str)
                
                ##Angle correction
                first_row_value_x = imu_go_kinect.loc[0,'Angle_X']
                first_row_value_y = imu_go_kinect.loc[0,'Angle_Y']
                first_row_value_z = imu_go_kinect.loc[0,'Angle_Z']

                imu_go_kinect.loc[:,'Angle_X'] = imu_go_kinect['Angle_X'] - first_row_value_x
                imu_go_kinect.loc[:,'Angle_Y'] = imu_go_kinect['Angle_Y'] - first_row_value_y
                imu_go_kinect.loc[:,'Angle_Z'] = imu_go_kinect['Angle_Z'] - first_row_value_z
                
                imu_go_kinect.to_csv("\\Data_new\\Processed\\IMU_2m\\"+t+'\\go\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)          
                
               
                #turn: start_turn ~ end_turn
                imu_turn = imu_new.copy()
                for i in range(0,len(imu_turn)):
                    timestamp2 = imu_turn.iloc[i,0]
                    if timestamp2 >= start_turn:
                        imu_turn.drop(imu_turn.head(i).index,inplace=True)
                        break
                imu_turn.reset_index(drop=True,inplace=True)
                for i in range(0,len(imu_turn)):
                    timestamp2 = imu_turn.iloc[i,0]
                    if timestamp2 > end_turn :
                        imu_turn.drop(imu_turn.tail(len(imu_turn)-i).index,inplace=True)
                        break
                imu_turn['Timestamp'] = imu_turn['Timestamp'].astype(str)
                if len(imu_turn)<2:
                    print("错误")                            

                first_row_value_x = imu_turn.loc[0,'Angle_X']
                first_row_value_y = imu_turn.loc[0,'Angle_Y']
                first_row_value_z = imu_turn.loc[0,'Angle_Z']
        
                imu_turn.loc[:,'Angle_X'] = imu_turn['Angle_X'] - first_row_value_x
                imu_turn.loc[:,'Angle_Y'] = imu_turn['Angle_Y'] - first_row_value_y
                imu_turn.loc[:,'Angle_Z'] = imu_turn['Angle_Z'] - first_row_value_z
                
                imu_turn.to_csv("\\Data_new\\Processed\\IMU_5m\\"+t+'\\turn\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False) 
                imu_turn.to_csv("\\Data_new\\Processed\\IMU_2m\\"+t+'\\turn\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)
                
                #back: end_turn ~ end
                imu_back = imu_new.copy()
                for i in range(0,len(imu_back)):
                    timestamp2 = imu_back.iloc[i,0]
                    if timestamp2 >= end_turn:
                        imu_back.drop(imu_back.head(i).index,inplace=True)
                        break
                imu_back.reset_index(drop=True,inplace=True)
                for i in range(0,len(imu_back)):
                    timestamp2 = imu_back.iloc[i,0]
                    if timestamp2 > end :
                        imu_back.drop(imu_back.tail(len(imu_back)-i).index,inplace=True)
                        break
                imu_back['Timestamp'] = imu_back['Timestamp'].astype(str)

                first_row_value_x = imu_back.loc[0,'Angle_X']
                first_row_value_y = imu_back.loc[0,'Angle_Y']
                first_row_value_z = imu_back.loc[0,'Angle_Z']
        
                imu_back.loc[:,'Angle_X'] = imu_back['Angle_X'] - first_row_value_x
                imu_back.loc[:,'Angle_Y'] = imu_back['Angle_Y'] - first_row_value_y
                imu_back.loc[:,'Angle_Z'] = imu_back['Angle_Z'] - first_row_value_z
                imu_back.to_csv("\\Data_new\\Processed\\IMU_5m\\"+t+'\\back\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False) 
                   
                
                #back_kinect: end_turn ~ end_kinect
                imu_back_kinect = imu_new.copy()
                for i in range(0,len(imu_back_kinect)):
                    timestamp2 = imu_back_kinect.iloc[i,0]
                    if timestamp2 >= end_turn:
                        imu_back_kinect.drop(imu_back_kinect.head(i).index,inplace=True)
                        break
                imu_back_kinect.reset_index(drop=True,inplace=True)
                for i in range(0,len(imu_back_kinect)):
                    timestamp2 = imu_back_kinect.iloc[i,0]
                    if timestamp2 > end_kinect :
                        imu_back_kinect.drop(imu_back_kinect.tail(len(imu_back_kinect)-i).index,inplace=True)
                        break
                imu_back_kinect['Timestamp'] = imu_back_kinect['Timestamp'].astype(str)
                
                first_row_value_x = imu_back_kinect.loc[0,'Angle_X']
                first_row_value_y = imu_back_kinect.loc[0,'Angle_Y']
                first_row_value_z = imu_back_kinect.loc[0,'Angle_Z']
        
                imu_back_kinect.loc[:,'Angle_X'] = imu_back_kinect['Angle_X'] - first_row_value_x
                imu_back_kinect.loc[:,'Angle_Y'] = imu_back_kinect['Angle_Y'] - first_row_value_y
                imu_back_kinect.loc[:,'Angle_Z'] = imu_back_kinect['Angle_Z'] - first_row_value_z
                
                imu_back_kinect.to_csv("\\Data_new\\Processed\\IMU_2m\\"+t+'\\back\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)                 
           
       
            

In [ ]:
imu_preprocess("\\Data_new\\Raw\\IMU")